In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [164]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/머신러닝/train.csv')
test = pd.read_csv('/content/drive/MyDrive/머신러닝/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/머신러닝/sample_submission.csv')

In [165]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
# 훈련 데이터 확인
train.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [167]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 15304 non-null  int64  
 1   gender             15304 non-null  object 
 2   age                15304 non-null  float64
 3   hypertension       15304 non-null  int64  
 4   heart_disease      15304 non-null  int64  
 5   ever_married       15304 non-null  object 
 6   work_type          15304 non-null  object 
 7   Residence_type     15304 non-null  object 
 8   avg_glucose_level  15304 non-null  float64
 9   bmi                15304 non-null  float64
 10  smoking_status     15304 non-null  object 
 11  stroke             15304 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 1.4+ MB


In [168]:
train.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,15304.000000,15304.000000,15304.000000,15304.000000,15304.000000,15304.000000,15304.000000
mean,7651.500000,41.417708,0.049726,0.023327,89.039853,28.112721,0.041296
std,4418.028595,21.444673,0.217384,0.150946,25.476102,6.722315,0.198981
min,0.000000,0.080000,0.000000,0.000000,55.220000,10.300000,0.000000
25%,3825.750000,26.000000,0.000000,0.000000,74.900000,23.500000,0.000000
50%,7651.500000,43.000000,0.000000,0.000000,85.120000,27.600000,0.000000
75%,11477.250000,57.000000,0.000000,0.000000,96.980000,32.000000,0.000000
max,15303.000000,82.000000,1.000000,1.000000,267.600000,80.100000,1.000000


In [169]:
train.shape, test.shape, submission.shape

((15304, 12), (10204, 11), (10204, 2))

In [170]:
train.head(4).T

,0,1,2,3
id,0,1,2,3
gender,Male,Male,Female,Male
age,28.0,33.0,42.0,56.0
hypertension,0,0,0,0
heart_disease,0,0,0,0
ever_married,Yes,Yes,Yes,Yes
work_type,Private,Private,Private,Private
Residence_type,Urban,Rural,Rural,Urban
avg_glucose_level,79.53,78.44,103.0,64.87
bmi,31.1,23.9,40.3,28.8


In [171]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [172]:
def chk_bmi(x):
  result = 7

  if x < 16:
    result = 0
  elif x < 17:
    result = 1
  elif x < 18.5:
    result = 2
  elif x < 25:
    result = 3
  elif x < 30:
    result = 4
  elif x < 35:
    result = 5
  elif x < 40:
    result = 6

  return result

In [173]:
def chk_age(x):
  result = 8

  if x < 10:
    result = 0
  elif x < 20:
    result = 1
  elif x < 30:
    result = 2
  elif x < 40:
    result = 3
  elif x < 50:
    result = 4
  elif x < 60:
    result = 5
  elif x < 70:
    result = 6
  elif x < 80:
    result = 7

  return result

In [174]:
# smoke
smoke_mapping = {'Unknown':0,'never smoked':1,'formerly smoked':2,'smokes':3}

# work - 스트레스 덜 받는 순?
work_mapping = {'children':0,'Govt_job':1,'Self-employed':2,'Private':3,'Never_worked':4}

In [175]:
# 성별
train['gender'] = train['gender'].apply(lambda x: 1 if x =='Female' else 0)
test['gender'] = test['gender'].apply(lambda x: 1 if x =='Female' else 0)

In [176]:
# 나이
train['age'] = train['age'].apply(chk_age)
test['age'] = test['age'].apply(chk_age)

In [177]:
# 결혼
train['ever_married'] = train['ever_married'].map({'Yes':0, 'No':1})
test['ever_married'] = test['ever_married'].map({'Yes':0, 'No':1})

In [178]:
# 일
train['work_type'] = train['work_type'].map(work_mapping)
test['work_type'] = test['work_type'].map(work_mapping)

In [179]:
# 주거
train['Residence_type'] = train['Residence_type'].map({'Urban':0, 'Rural':1})
test['Residence_type'] = test['Residence_type'].map({'Urban':0, 'Rural':1})

In [180]:
# 혈당
train['avg_glucose_level'] = train['avg_glucose_level'].apply(lambda x: 1 if x> 100 else 0)
test['avg_glucose_level'] = test['avg_glucose_level'].apply(lambda x: 1 if x> 100 else 0)

In [181]:
# bmi
train['bmi'] = train['bmi'].apply(chk_bmi)
test['bmi'] = test['bmi'].apply(chk_bmi)

In [182]:
# 흡연
train['smoking_status'] = train['smoking_status'].map(smoke_mapping)
test['smoking_status'] = test['smoking_status'].map(smoke_mapping)

In [183]:
train = pd.concat([train, test],ignore_index=True)
train.shape

(25508, 11)

In [184]:
X_sub = train[pd.isnull(train['stroke'])]  # 정답 없는 것들
X_sub.shape

(10204, 11)

In [185]:
X = train[~pd.isnull(train['stroke'])]  # 정답 있는 것들
X.shape

(15304, 11)

In [186]:
y = X['stroke']
X = X.drop(['stroke'],axis=1)
X_sub = X_sub.drop(['stroke'], axis=1)

In [187]:
X.shape, y.shape, X_sub.shape

((15304, 10), (15304,), (10204, 10))

In [188]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [189]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.15,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13008, 10), (2296, 10), (13008,), (2296,))

In [191]:
grid_params = {
    'C'           :[0.001, 0.01, 0.1, 0.125, 1], # 과적합 방지 규제 강도, 높을수록 강하다. L1, L2
    'max_iter'    :[600, 700, 800, 900 ], # 모델 성능을 높이기 위해 학습의 반복 회수
    'solver'     :['liblinear'],        # 과적합 방지용 L1, L2 모두 지원
    'random_state':[0, 42, 100]     # 난수시드
}

In [192]:
clf = LogisticRegression()

In [193]:
model = GridSearchCV( clf, grid_params, scoring='roc_auc', cv=5)

In [194]:
model.fit( X_train, y_train )

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 0.125, 1],
                         'max_iter': [600, 700, 800, 900],
                         'random_state': [0, 42, 100],
                         'solver': ['liblinear']},
             scoring='roc_auc')

In [195]:
model.best_params_, model.best_score_

({'C': 1, 'max_iter': 600, 'random_state': 0, 'solver': 'liblinear'},
 0.8790768040778326)

In [196]:
from sklearn.metrics import roc_auc_score

y_pred = model.best_estimator_.predict_proba( X_test )[ :, 1]

roc_auc_score(y_test,y_pred) # 모델 성능값과 비교해서 과적합 판단

0.8835218441378321

In [197]:
submission['stroke'] = model.best_estimator_.predict_proba( X_sub )[ :, 1]
# kaggle에 제출할 파일 생성
submission.to_csv( 'mlpro_team_logi.csv', index=False )

In [198]:
clf = DecisionTreeClassifier()

In [199]:
grid_params2 = {
    'max_features': ['auto', 'sqrt', 'log2'],
    'ccp_alpha': [0.1, .01, .001],
    'max_depth' : [5, 6, 7, 8, 9],
    'criterion' :['gini', 'entropy']
}

In [200]:
model = GridSearchCV( clf, grid_params2, scoring='roc_auc', cv=5)
model.fit(X_train,y_train)
model.best_params_, model.best_score_

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

({'ccp_alpha': 0.001,
  'criterion': 'entropy',
  'max_depth': 6,
  'max_features': 'auto'},
 0.8496194578113428)

In [201]:
from sklearn.metrics import roc_auc_score

y_pred = model.best_estimator_.predict_proba( X_test )[ :, 1]

roc_auc_score(y_test,y_pred) # 모델 성능값과 비교해서 과적합 판단

0.8458451899854134

In [202]:
submission['stroke'] = model.best_estimator_.predict_proba( X_sub )[ :, 1]
# kaggle에 제출할 파일 생성
submission.to_csv( 'mlpro_team_deci.csv', index=False )

In [203]:
clf = RandomForestClassifier()

In [204]:
grid_params3 = {
    'n_estimators' : [10, 100],
    'max_depth' : [6, 8, 10, 12],
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

In [205]:
model = GridSearchCV( clf, grid_params3, scoring='roc_auc', cv=5)
model.fit( X_train, y_train )
model.best_params_, model.best_score_

({'max_depth': 6,
  'min_samples_leaf': 12,
  'min_samples_split': 16,
  'n_estimators': 100},
 0.8787702601592553)

In [206]:
from sklearn.metrics import roc_auc_score

y_pred = model.best_estimator_.predict_proba( X_test )[ :, 1]

roc_auc_score(y_test,y_pred) # 모델 성능값과 비교해서 과적합 판단

0.8927760109041345

In [207]:
submission['stroke'] = model.best_estimator_.predict_proba( X_sub )[ :, 1]
# kaggle에 제출할 파일 생성
submission.to_csv( 'mlpro_team_rf.csv', index=False )